In [ ]:
!pip install -q numpy pandas transformers torch scikit-learn nltk googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.2.3 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
openai 1.57.4 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.


In [ ]:
import torch
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [ ]:
import pandas as pd

df = pd.read_csv('good_train.csv')

In [ ]:
validation = df[2550:].copy()
df = df[:2550]

In [ ]:
# from transformers import DebertaTokenizer, DebertaModel
# tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
import random
from nltk.corpus import wordnet
from googletrans import Translator
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# 1. Synonym Replacement
def synonym_replacement(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()

    for  _ in range(n):
        word = random.choice(words)
        synonyms = set()

        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())

        if synonyms:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if w == word else w for w in new_words]

    return ' '.join(new_words)

# 2. Random Insertion
def random_insertion(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()

    for _ in range(n):
        word = random.choice(words)
        synonyms = set()

        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())

        if synonyms:
            synonym = random.choice(list(synonyms))
            insert_position = random.randint(0, len(new_words) - 1)
            new_words.insert(insert_position, synonym)

    return ' '.join(new_words)

# 3. Random Swap
def random_swap(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()

    for _ in range(n):
        idx1 = random.randint(0, len(words) - 1)
        idx2 = random.randint(0, len(words) - 1)

        new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]

    return ' '.join(new_words)

# 4. Random Deletion
def random_deletion(sentence, p=0.2):
    words = sentence.split()
    if len(words) == 1:  # We cannot delete the only word in the sentence
        return sentence

    new_words = [word for word in words if random.uniform(0, 1) > p]

    if len(new_words) == 0:
        return random.choice(words)  # Return a random word if all are deleted

    return ' '.join(new_words)

# 5. Back Translation (Translation to another language and then back to the original)
def back_translation(sentence, src_lang='en', tgt_lang='fr'):
    translator = Translator()
    translated = translator.translate(sentence, src=src_lang, dest=tgt_lang).text
    back_translated = translator.translate(translated, src=tgt_lang, dest=src_lang).text
    return back_translated

def EDA(sentence):
    sentence = synonym_replacement(sentence, n=2)
    sentence = random_insertion(sentence, n=2)
    sentence = random_swap(sentence, n=2)
    sentence = random_deletion(sentence, p=0.2)
    # sentence = back_translation(sentence, src_lang='en', tgt_lang='fr')

    return sentence

def EDA2(sentence):
    sentence = synonym_replacement(sentence, n=3)
    sentence = random_insertion(sentence, n=3)
    sentence = random_swap(sentence, n=3)
    sentence = random_deletion(sentence, p=0.3)
    # sentence = back_translation(sentence, src_lang='en', tgt_lang='fr')

    return sentence

def EDA3(sentence):
    sentence = synonym_replacement(sentence, n=1)
    sentence = random_insertion(sentence, n=1)
    sentence = random_swap(sentence, n=1)
    sentence = random_deletion(sentence, p=0.1)
    # sentence = back_translation(sentence, src_lang='en', tgt_lang='fr')

    return sentence

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
z1 = df.copy()
z1['text'] = z1['text'].apply(EDA)

df = pd.concat([df, z1], ignore_index=True)

encoded = df['text'].apply(lambda x: tokenizer.encode_plus(
    x,
    max_length=64,
    padding='max_length',
    truncation=True,
    return_attention_mask=True
))


df['input_ids'] = encoded.apply(lambda x: x['input_ids'])
df['attention_mask'] = encoded.apply(lambda x: x['attention_mask'])



encoded = validation['text'].apply(lambda x: tokenizer.encode_plus(
    x,
    max_length=64,
    padding='max_length',
    truncation=True,
    return_attention_mask=True
))


validation['input_ids'] = encoded.apply(lambda x: x['input_ids'])
validation['attention_mask'] = encoded.apply(lambda x: x['attention_mask'])

In [ ]:
go_emo = pd.read_csv('1.csv')
non_zero_emotions = go_emo[['text', 'anger', 'fear', 'joy', 'sadness', 'surprise']]

non_zero_emotions = non_zero_emotions[
    (go_emo['anger'] != 0) |
    (go_emo['fear'] != 0) |
    (go_emo['joy'] != 0) |
    (go_emo['sadness'] != 0) |
    (go_emo['surprise'] != 0)
]


# Get 500 random rows where all emotions are zero
zero_emotions = go_emo[
    (go_emo['anger'] == 0) &
    (go_emo['fear'] == 0) &
    (go_emo['joy'] == 0) &
    (go_emo['sadness'] == 0) &
    (go_emo['surprise'] == 0)
].sample(n=150, random_state=42)

zero_emotions = zero_emotions[['text', 'anger', 'fear', 'joy', 'sadness', 'surprise']]

# Concatenate the two dataframes
go_emo = pd.concat([non_zero_emotions, zero_emotions])

In [ ]:
encoded = go_emo['text'].apply(lambda x: tokenizer.encode_plus(
    x,
    max_length=64,
    padding='max_length',
    truncation=True,
    return_attention_mask=True
))


go_emo['input_ids'] = encoded.apply(lambda x: x['input_ids'])
go_emo['attention_mask'] = encoded.apply(lambda x: x['attention_mask'])

In [ ]:
emotions = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotions2 = ['anger', 'fear', 'joy', 'sadness', 'surprise']

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split

class TweetsDataset(Dataset):

    def __init__(self, df, labels):
        self.df = df.copy()
        self.labels = labels

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx, :]

        return {
            'input_ids': torch.tensor(row['input_ids']),
            'attention_mask': torch.tensor(row['attention_mask']),
            'labels': torch.tensor(row[self.labels].tolist(), dtype=torch.float32)
        }

train_dataset = TweetsDataset(df, emotions2)
val_dataset = TweetsDataset(validation, emotions2)
go_emo_dataset = TweetsDataset(go_emo, emotions2)

# train_size = int(len(df) * .8)
# val_size = len(df) - train_size

# train_dataset, val_dataset = random_split(
#     dataset,
#     [train_size, val_size],
#     generator=torch.Generator().manual_seed(42)
# )

train_dataloader = DataLoader(train_dataset, batch_size=16)
val_dataloader = DataLoader(val_dataset, batch_size=16)
go_dataloader = DataLoader(go_emo_dataset, batch_size=16)

# full_dataloader = DataLoader(train_dataset, batch_size=16)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class TweetEmotionClassifier(nn.Module):

    def __init__(self):
        super().__init__()

        self.bert = AutoModel.from_pretrained("microsoft/deberta-v3-base")
        self.layer = nn.LayerNorm(768)
        self.linear = nn.Linear(768, 5)

        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]

        # h = self.layer(cls_output)
        h = self.dropout(cls_output)
        h = self.linear(h)
        # h = F.relu(h)

        # h = self.dropout(h)
        # out = self.linear2(h)

        return h

In [ ]:
model = TweetEmotionClassifier()
model = model.to(device)

optimizer = torch.optim.AdamW([
    {'params': model.bert.parameters(), 'lr': 2e-5},
    {'params': list(model.linear.parameters()), 'lr': 1e-3},
])

loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
from sklearn.metrics import f1_score

for epoch in range(3):
    model.train()

    # for param in model.bert.encoder.layer[12 - epoch - 1].parameters():
    #         param.requires_grad = True

    for idx, batch in enumerate(go_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        pred = model(input_ids, attention_mask)

        loss = loss_fn(pred, labels)

        loss.backward()
        optimizer.step()

        # if idx % 50 == 0:
        #     print(loss.item())

    # model.eval()
    # all_preds = []
    # all_labels = []
    # with torch.no_grad():
    #     for idx, batch in enumerate(val_dataloader):
    #         input_ids = batch['input_ids'].to(device)
    #         attention_mask = batch['attention_mask'].to(device)
    #         labels = batch['labels']

    #         preds = model(input_ids, attention_mask)
    #         preds = preds.cpu().tolist()
    #         preds = [1 if i > 0.5 else 0 for b in preds for i in b]

    #         labels = [j for i in labels for j in i]
    #         all_preds.extend(preds)
    #         all_labels.extend(labels)

    # s = f1_score(all_labels, all_preds, average='macro')
    # print(f'f1_score={s}')


In [ ]:
from sklearn.metrics import f1_score

for epoch in range(4):
    model.train()

    for param in model.bert.encoder.layer[:6].parameters():
            param.requires_grad = False

    for idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        pred = model(input_ids, attention_mask)

        loss = loss_fn(pred, labels)

        loss.backward()
        optimizer.step()

        # if idx % 50 == 0:
        #     print(loss.item())

    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for idx, batch in enumerate(val_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']

            preds = model(input_ids, attention_mask)
            preds = preds.cpu().tolist()
            preds = [1 if i > 0.5 else 0 for b in preds for i in b]

            labels = [j for i in labels for j in i]
            all_preds.extend(preds)
            all_labels.extend(labels)

    s = f1_score(all_labels, all_preds, average='macro')
    print(f'f1_score={s}')


f1_score=0.8085770681574163
f1_score=0.8334599829402474
f1_score=0.837209020711146
f1_score=0.8190136674988797


In [ ]:
test_df = pd.read_csv('test.csv')


In [ ]:
torch.save(model, 'large1.pth')

KeyboardInterrupt: 

In [ ]:
encoded = test_df['text'].apply(lambda x: tokenizer.encode_plus(
    x,
    max_length=64,
    padding='max_length',
    truncation=True,
    return_attention_mask=True
))
# test_df['text'].apply(clean_text)

test_df['input_ids'] = encoded.apply(lambda x: x['input_ids'])
test_df['attention_mask'] = encoded.apply(lambda x: x['attention_mask'])

test_dataset = TweetsDataset(test_df, emotions2)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=1
)

In [ ]:
model.eval()

qk = []

with torch.no_grad():
    for idx, batch in enumerate(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels']

        # pp = []
        # for model in models:
        ahah = model(input_ids, attention_mask).squeeze(0)
        #     pp.append(temp)
        # ahah = torch.mean(torch.stack(pp), dim=0)
        preds = (ahah > 0.5)
        preds = preds.int()
        # print(preds)
        # break
        bb = test_df.loc[idx, 'id']

        qk.append([bb] + preds.tolist())

output_df = pd.DataFrame(qk, columns=['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])


In [ ]:
output_df.to_csv('pred_eng.csv', index=False)
from google.colab import files
files.download('pred_eng.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
output_df